In [36]:
import os


In [39]:
%pwd

'f:\\Complete-MLOps-Bootcamp-by-Krish-Naik'

In [40]:
os.chdir("./data-science-project")
%pwd

'f:\\Complete-MLOps-Bootcamp-by-Krish-Naik\\data-science-project'

In [41]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    """
    Configuration for data ingestion.
    """
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [42]:
from src.data_science_project.constants import *
from src.data_science_project.utils.common import read_yaml, create_directories


In [43]:
class ConfigurationManager:
    """
    Configuration manager to handle configurations.
    """
    def __init__(self, config_file_path = CONFIG_FILE_PATH, params_file_path = PARAMS_FILE_PATH, schema_file_path = SCHEMA_FILE_PATH):
    
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """
        Returns the data ingestion configuration.
        """
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [44]:
import os
import urllib.request as request
from src.data_science_project import logger
import zipfile

In [45]:
## component for data ingestion
class DataIngestion:
    """
    Data ingestion component to handle data downloading and extraction.
    """
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self):
        """
        Download data from the source URL.
        """
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"Downloaded file: {filename} with headers:\n {headers}")

        else:
            logger.info(f"File already exists: {self.config.local_data_file}. Skipping download.")

    def extract_data(self):
        """
        Extract downloaded data.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted data to: {unzip_path}")

In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_data()
except Exception as e:
    logger.exception(f"An error occurred during data ingestion: {e}")
    raise e

[2025-06-22 02:07:55,860: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-22 02:07:55,861: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-22 02:07:55,862: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-22 02:07:55,862: INFO: common: created directory at: artifacts]
[2025-06-22 02:07:55,863: INFO: common: created directory at: artifacts/data_ingestion]
[2025-06-22 02:07:55,863: INFO: 3699610440: File already exists: artifacts/data_ingestion/data.zip. Skipping download.]
[2025-06-22 02:07:55,864: INFO: 3699610440: Extracted data to: artifacts/data_ingestion]


: 